# Matching

- This is a concept that involves finding two or more pieces of data that are same.

- This can be dne by looking at the values of data or looking at the relationship between data.

- Imagine you have a box of toys. You want to find all of the pairs of matching toys.

- You can do this by matching the toys by their color, size, or shape. For example, you could find all of the red toys, or all of the big toys, or all of the round toys.

- Once you have found all of the matching toys, you can put them away in pairs.

# Case Study: Are Catholic Schools Better?

In [1]:
!pip install CausalInference

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.5 MB/s eta 0:00:00


In [2]:
# Loading the directory
%cd /content/drive/MyDrive/Business Analyst Workbooks/Matching

/content/drive/MyDrive/Business Analyst Workbooks/Matching


In [3]:
# Import the Libraries
import pandas as pd
import numpy as np
import seaborn as sns
import scipy.stats as ss
from causalinference import CausalModel

In [4]:
# Get Data
df = pd.read_csv("school.csv")

In [5]:
df.head()

,childid,catholic,race,number_places_lived,mom_age,dad_age,dad_education,mom_education,mom_score,dad_score,income,poverty,food_stamps,score_standardized
0,0001002C,0,"WHITE, NON-HISPANIC",1,47,45,DOCTORATE OR PROFESSIONAL DEGREE,SOME COLLEGE,53.50,77.5,62500.5,0,0,0.981753
1,0001004C,0,"WHITE, NON-HISPANIC",1,41,48,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,34.95,53.5,45000.5,0,0,0.594378
2,0001010C,0,"WHITE, NON-HISPANIC",1,43,55,"MASTER'S DEGREE (MA, MS)",GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,63.43,53.5,62500.5,0,0,0.490611
3,0001011C,1,"WHITE, NON-HISPANIC",1,38,39,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,53.50,53.5,87500.5,0,0,1.451278
4,0001012C,0,"WHITE, NON-HISPANIC",1,47,57,DOCTORATE OR PROFESSIONAL DEGREE,"MASTER'S DEGREE (MA, MS)",61.56,77.5,150000.5,0,0,2.595699


In [7]:
# Drop the unecessary data
df = df.iloc[:,1:]
df.head()

,catholic,race,number_places_lived,mom_age,dad_age,dad_education,mom_education,mom_score,dad_score,income,poverty,food_stamps,score_standardized
0,0,"WHITE, NON-HISPANIC",1,47,45,DOCTORATE OR PROFESSIONAL DEGREE,SOME COLLEGE,53.50,77.5,62500.5,0,0,0.981753
1,0,"WHITE, NON-HISPANIC",1,41,48,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,34.95,53.5,45000.5,0,0,0.594378
2,0,"WHITE, NON-HISPANIC",1,43,55,"MASTER'S DEGREE (MA, MS)",GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,63.43,53.5,62500.5,0,0,0.490611
3,1,"WHITE, NON-HISPANIC",1,38,39,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,53.50,53.5,87500.5,0,0,1.451278
4,0,"WHITE, NON-HISPANIC",1,47,57,DOCTORATE OR PROFESSIONAL DEGREE,"MASTER'S DEGREE (MA, MS)",61.56,77.5,150000.5,0,0,2.595699


# Unconfoundedness

- Imagine you have two groups of toys: one group of red toys, and one group of blue toys. You want to know if being red makes a toy more fun to play with.

- To find out, you could look at how much fun children have playing with red toys versus blue toys. But there is a problem: some toys might be more fun to play with for other reasons, such as because they are shaped like a dinosaur or because they make a noise.

- To solve this problem, you need to make sure that the two groups of toys are similar in all other ways, except for their color. This is called unconfoundedness.

- One way to make sure that the two groups of toys are unconfounded is to match them. This means finding two toys that are similar in all other ways, except for their color. You can match the toys by looking at their size, shape, and other factors.

- Once you have matched the toys, you can compare how much fun children have playing with them. If the red toys are more fun to play with than the blue toys, then you can conclude that being red makes a toy more fun to play with.

- Unconfoundedness is an important concept in data analytics. It allows us to draw causal conclusions from data. This means that we can learn about the causes and effects of things.


# Data Analysis

- With the help of groupby function we need to group the functions through which they are catholic or not and the mean of all features are determined using mean function.

In [8]:
# Comparing Group Avg's
df.groupby('catholic').mean()

<ipython-input-8-c7c30d618864>:2: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df.groupby('catholic').mean()


,number_places_lived,mom_age,dad_age,mom_score,dad_score,income,poverty,food_stamps,score_standardized
catholic,,,,,,,,,
0,1.106246,37.794621,40.134919,43.909495,42.590520,65393.928540,0.101578,0.045566,0.163128
1,1.073118,39.775269,42.007527,47.620871,45.908269,86180.625269,0.016129,0.006452,0.219685


In [10]:
 # T-Test (Done for Continuous Variables)
group1 = df.where(df.catholic == 0).dropna()["income"] # Non-Catholic
group2 = df.where(df.catholic == 1).dropna()["income"] # Catholic
group2.head()

3      87500.5
21    150000.5
22     87500.5
23    150000.5
24    150000.5
Name: income, dtype: float64

In [12]:
stat, p = ss.ttest_ind(group1,group2)
print(p)

5.943636213205364e-41


- We will be creating a loop that will check all the entries and find out the p-value of the variable.

In [13]:
# What to look
continuous = ["number_places_lived", "mom_age", "dad_age", "mom_score", "dad_score", "income"]
# Where to store the result
stat = {}
p = {}
# Loop
for x in continuous:
  group1 = df.where(df.catholic == 0).dropna()[x] # Non-Catholic
  group2 = df.where(df.catholic == 1).dropna()[x] # Catholic
  stat[x], p[x] = ss.ttest_ind(group1,group2)
ttests = pd.DataFrame.from_dict(p, orient="Index")
ttests.columns = ['pvalue']
print(ttests)

                           pvalue
number_places_lived  7.072609e-03
mom_age              1.359492e-22
dad_age              3.344265e-16
mom_score            2.280116e-19
dad_score            5.489722e-18
income               5.943636e-41


- We have studied our continuous variables and the p-value concludes that these can undergo matching process.|

- We will do Chi-Square test do determine the relationship between binary variables.

In [14]:
# Chi-Square Test
tab = pd.crosstab(index = df.poverty,
                  columns = df.catholic)
tab

catholic,0,1
poverty,,
0,4042,915
1,457,15


In [17]:
statistic, p, dof, exp = ss.chi2_contingency(tab)
p

6.511354893726035e-17

- We will create a loop for chi square values too.

In [21]:
from pandas.core.arrays.categorical import CategoricalT
# What to loop
categorical = ["poverty", "food_stamps"]
# Where to Store
statistic = {}
p = {}
dof = {}
exp = {}
# Loop
for x in categorical:
  tab = pd.crosstab(index = df[x],
                  columns = df.catholic)
  statistic[x], p[x], dof[x], exp[x] = ss.chi2_contingency(tab)
chisquare = pd.DataFrame.from_dict(p, orient = "Index")
chisquare.columns = ["pvalue"]
print(chisquare)

                   pvalue
poverty      6.511355e-17
food_stamps  3.294153e-08


- The p-values are very low so there is strong relation between binary variables too.

# Matching Preparation

In [24]:
temp = pd.get_dummies(df)
temp.head(1)

,catholic,number_places_lived,mom_age,dad_age,mom_score,dad_score,income,poverty,food_stamps,score_standardized,...,dad_education_VOC/TECH PROGRAM,mom_education_8TH GRADE OR BELOW,mom_education_9TH - 12TH GRADE,mom_education_BACHELOR'S DEGREE,mom_education_DOCTORATE OR PROFESSIONAL DEGREE,mom_education_GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,mom_education_HIGH SCHOOL DIPLOMA/EQUIVALENT,"mom_education_MASTER'S DEGREE (MA, MS)",mom_education_SOME COLLEGE,mom_education_VOC/TECH PROGRAM
0,0,1,47,45,53.5,77.5,62500.5,0,0,0.981753,...,0,0,0,0,0,0,0,0,1,0


In [28]:
print(temp.columns)

Index(['catholic', 'number_places_lived', 'mom_age', 'dad_age', 'mom_score',
       'dad_score', 'income', 'poverty', 'food_stamps', 'score_standardized',
       'race_AMERICAN INDIAN OR ALASKA NATIVE', 'race_ASIAN',
       'race_BLACK OR AFRICAN AMERICAN, NON-HISPANIC',
       'race_HISPANIC, RACE NOT SPECIFIED', 'race_HISPANIC, RACE SPECIFIED',
       'race_MORE THAN ONE RACE, NON HISPANIC',
       'race_NATIVE HAWAIIAN, OTHER PACIFIC ISLANDER', 'race_NOT ASCERTAINED',
       'race_WHITE, NON-HISPANIC', 'dad_education_8TH GRADE OR BELOW',
       'dad_education_9TH - 12TH GRADE', 'dad_education_BACHELOR'S DEGREE',
       'dad_education_DOCTORATE OR PROFESSIONAL DEGREE',
       'dad_education_GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE',
       'dad_education_HIGH SCHOOL DIPLOMA/EQUIVALENT',
       'dad_education_MASTER'S DEGREE (MA, MS)', 'dad_education_SOME COLLEGE',
       'dad_education_VOC/TECH PROGRAM', 'mom_education_8TH GRADE OR BELOW',
       'mom_education_9TH - 12TH GRADE', 'mom_

# Curse of Dimensionality

- We need to keep the model as simple as possible, let'say you have a variable with three options and another variable with 3 more options, this leads to exponential observations so this leads to artificial and complicated outcomes.

In [25]:
df.race.value_counts()

WHITE, NON-HISPANIC                        3654
HISPANIC, RACE NOT SPECIFIED                408
HISPANIC, RACE SPECIFIED                    387
BLACK OR AFRICAN AMERICAN, NON-HISPANIC     357
ASIAN                                       342
MORE THAN ONE RACE, NON HISPANIC            123
NATIVE HAWAIIAN, OTHER PACIFIC ISLANDER      93
AMERICAN INDIAN OR ALASKA NATIVE             62
NOT ASCERTAINED                               3
Name: race, dtype: int64

- As this is very complicated, we need to see to simplify,

- Consider White race, Group Hispanic, consider Black or African American and Asian. Then, exclude the rest from the list.

In [35]:
# Preparing Race Variable with if else condition
df['race_asian'] = np.where(df.race == 'ASIAN', 1, 0)
df['race_white'] = np.where(df.race == 'WHITE, NON-HISPANIC', 1, 0)
df['race_black'] = np.where(df.race == 'BLACK OR AFRICAN AMERICAN, NON-HISPANIC', 1, 0)
df['race_hispanic'] = np.where((df.race == 'HISPANIC, RACE NOT SPECIFIED') |
                                 (df.race == 'HISPANIC, RACE SPECIFIED' ), 1, 0)
df.head()

,catholic,race,number_places_lived,mom_age,dad_age,dad_education,mom_education,mom_score,dad_score,income,poverty,food_stamps,score_standardized,race_white,race_hispanic,race_black,race_asian
0,0,"WHITE, NON-HISPANIC",1,47,45,DOCTORATE OR PROFESSIONAL DEGREE,SOME COLLEGE,53.50,77.5,62500.5,0,0,0.981753,1,0,0,0
1,0,"WHITE, NON-HISPANIC",1,41,48,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,34.95,53.5,45000.5,0,0,0.594378,1,0,0,0
2,0,"WHITE, NON-HISPANIC",1,43,55,"MASTER'S DEGREE (MA, MS)",GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,63.43,53.5,62500.5,0,0,0.490611,1,0,0,0
3,1,"WHITE, NON-HISPANIC",1,38,39,BACHELOR'S DEGREE,GRADUATE/PROFESSIONAL SCHOOL-NO DEGREE,53.50,53.5,87500.5,0,0,1.451278,1,0,0,0
4,0,"WHITE, NON-HISPANIC",1,47,57,DOCTORATE OR PROFESSIONAL DEGREE,"MASTER'S DEGREE (MA, MS)",61.56,77.5,150000.5,0,0,2.595699,1,0,0,0
